# Plot COS FUV dark counts as a function of time
Author: Jo Taylor

Date: June 26, 2020

In [ ]:
import datashader as ds
import holoviews as hv
from holoviews.operation.datashader import datashade, dynspread
from holoviews import opts
import panel as pn
hv.extension('bokeh')

from query_darks import all_darks

### Get the dark values for all PHAs from the database

In [ ]:
# Get dark values from Darks database
# This could take several seconds
results = all_darks()

In [ ]:
# Sort results by date
results.sort_values(by=["expstart"], inplace=True)
#results["solar_flux"] = results["solar_flux"] - 100

# Can use dask dataframe if desired
#from dask import dataframe as dd
#import multiprocessing as mp
#results = dd.from_pandas(results, npartitions=mp.cpu_count())
#results.persist()

### Create the plots for dark counts and solar flux vs time

In [ ]:
# Create dictionary with a Curve element for each PHA plot, then collect into a HoloMap to get a PHA slider widget
curves = {i: hv.Curve(results, "expstart", f"dark_pha{i}") for i in range(32)}
pha_holomap = hv.HoloMap(curves, kdims="PHA")

In [ ]:
# Make a Curve element for the solar flux
solar = hv.Curve(results, "expstart", "solar_flux")

In [ ]:
# Use dynspread to make line widths larger
pha_spread = dynspread(datashade(pha_holomap, cmap=["cornflowerblue"]), threshold=.3)\
                    .opts(title="Darks vs Time", xlabel="Date [MJD]", ylabel="Dark Counts")\
                    .redim.range(expstart=(55000,59000), dark_pha0=(0,500))
solar_spread = dynspread(datashade(solar, cmap=["mediumvioletred"]), threshold=.3)\
                    .opts(xlabel="Date [MJD]", ylabel="Solar Flux")
# Create legend for Darks and Solar curves
legend = hv.NdOverlay({"Darks": hv.Points([[57000,0]], label="Darks").opts(color="cornflowerblue", size=0),
                        "Solar Flux": hv.Points([[57000,0]], label="Solar Flux").opts(color="mediumvioletred", size=0)},
                     kdims="Element")
# Put slider on top in case not plotting in a panel
hv.output(widget_location="top")

In [ ]:
# Partition Elements into panels so that widget can be moved around at will
pha_panel = pn.panel(pha_spread.opts(height=400, width=900))
#pha_panel = pn.panel(pha_spread*legend.opts(height=400, width=900)) # Legend not used at the moment
pha_plot = pha_panel[0]
pha_slider = pha_panel[1]
solar_panel = pn.panel(solar_spread.opts(height=200, width=900))
layout = pn.Column(pha_slider, pha_plot, solar_panel)

### Display plot
Use the slider widget to see the dark counts as a function of recorded PHA. Solar flux is plotted underneath. Bokeh plot tools to the right of each plot can pan and zoom as desired. Click the circle arrow symbol to reset plot to original.

_Note: depending on system memory, there may be a slight delay between moving slider and plot update._

In [ ]:
# Display final plot 
layout

### Plot one PHA or a sum of PHAs at a time
To do so, execute `plot_pha()` with either one or multiple PHA values:
```
plot_pha(10)
```
or
```
plot_pha([1, 11, 21])
```
If multiple PHAs are supplied, _**the sum of their individual arrays will be displayed**_.

Choose to overlay the solar flux by toggling the `showsolar` switch.

In [ ]:
# Convenience function to plot one or more PHA curves at a time
def plot_pha(pha, showsolar=True):
    if isinstance(pha, list):
        cols = [f"dark_pha{i}" for i in pha]
        results["sumpha"] = results[cols].sum(axis=1)
        curve = hv.Curve(results, "expstart", "sumpha")
    else: 
        curve = pha_holomap[pha]
    pha_spread = dynspread(datashade(curve, cmap=["cornflowerblue"]), threshold=.3)\
    .opts(title=f"Darks vs Time for PHAs {pha}", xlabel="Date [MJD]", ylabel="Dark Counts")
    legend = hv.NdOverlay({"Darks": hv.Points([[57000,0]], label="Darks").opts(color="cornflowerblue", size=0),
                        "Solar Flux": hv.Points([[57000,0]], label="Solar Flux").opts(color="mediumvioletred", size=0)},
                        kdims="Element")
 
    if showsolar is True:
        solar_spread = dynspread(datashade(solar, cmap=["mediumvioletred"]).redim.range(x=(55000,59000), y=(0,100)), threshold=.3)
        layout = pha_spread * solar_spread * legend
    else:
        layout = pha_spread
    layout.opts(opts.RGB(height=400, width=950))
    
    return layout

In [ ]:
plot_pha([1, 11, 21], showsolar=True)

### Create the plots for dark counts vs solar flux

In [ ]:
# Create dictionary with a Scatter element for each PHA plot, then collect into a HoloMap to get a PHA slider widget
scatters = {i: hv.Scatter(results, "solar_flux", f"dark_pha{i}") for i in range(32)}
pha_holomap2 = hv.HoloMap(scatters, kdims="PHA")
# Use dynspread to make line widths larger
pha_spread2 = dynspread(datashade(pha_holomap2, cmap=["cornflowerblue"]), threshold=.3)\
                    .opts(title="Darks vs Solar Flux", xlabel="Solar Flux", ylabel="Dark Counts")\
                    .redim.range(expstart=(60,220), dark_pha0=(0,500))
layout2 = pha_spread2
layout2.opts(opts.RGB(height=400, width=950))
# Put slider on top in case not plotting in a panel
hv.output(widget_location="top")

In [ ]:
layout